In [ ]:
%matplotlib inline

# system imports
import matplotlib
matplotlib.use('Agg')
from matplotlib import pylab as plt
from scipy.optimize import leastsq
import numpy
import os

# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afwDisplay
import lsst.afw.display.utils as afwDisplayUtils
import lsst.afw.display.utils as afwDisplayUtils
from lsst.ip.isr import IsrTask
import lsst.afw.detection as afwDetection


# Firefly client imports
from firefly_client import FireflyClient
from IPython.display import IFrame

In [ ]:
isr_config = IsrTask.ConfigClass()

isr_config.doDark=False
isr_config.doFlat=False
isr_config.doFringe=False
isr_config.doDefect=False
isr_config.doAddDistortionModel=False
isr_config.doLinearize=False

In [ ]:
isr = IsrTask(config=isr_config)

In [ ]:
BOOTCAMP_REPO_DIR= '/project/bootcamp/repo_RTM-007/'
butler = Butler(BOOTCAMP_REPO_DIR)
print(butler.queryMetadata('raw', ['visit'], dataId={'imageType': 'FE55', 'testType': 'FE55'}))

In [ ]:
dId = {'visit': 258334666, 'detector': 2}
raw = butler.get('raw', **dId)
bias = butler.get('bias', **dId)

In [ ]:
result = isr.run(raw, bias=bias)

In [ ]:
detector = result.exposure.getDetector()
amp = detector[1]

masked_img = result.exposure.getMaskedImage()
work_img = masked_img[amp.getBBox()]
arr = work_img.getImage().getArray()
    
bkgd = numpy.median(arr) # estimate a background and subtract it
arr -= bkgd

thresh = afwDetection.Threshold(100)
fs = afwDetection.FootprintSet(work_img, thresh) # detect hits
fs = afwDetection.FootprintSet(fs, 1, False) # grow the detection footprints
fs.makeHeavy(work_img)

foots = fs.getFootprints()

In [ ]:
len(foots)

In [ ]:
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
# Unfortunately, afwDisplayUtils.drawFootprint does not accept an `alpha` kwarg, so we cannot control
# the transparency of the footprints being drawn

afwDisplay.setDefaultMaskTransparency(0.5)
display = afwDisplay.Display()
display.scale('asinh', 'zscale')
display.mtv(work_img)
for fs in foots:
    afwDisplayUtils.drawFootprint(fs,display=display)
plt.rcParams['figure.figsize']=[50.0,50.0]
plt.show()

In [ ]:
help(afwDisplayUtils.drawFootprint)